# Исследование состояния выявления и лечения злокачественных образований в России

Заказчик из Министерства здравоохранения России ставит задачу:
- создать универсальный обработчик, который будет обрабатывать разрозненные файлы, предоставленых в формате .xls, без участия человека и сохранять обработанный результат в том или ином виде: обработанной сводной таблицы
- произвести анализ обработанных данных

Нам предоставлены файлы, содержащие в себе данные за 2021 год:
- Сведения о лечении злокачественных новообразований (ЗНО), впервые зарегистрированных в 2021 г. подлежащих радикальному лечению
- Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г.
- Заболеваемость населения территорий России злокачественными новообразованиями
- Смертность населения территорий России от злокачественных новообразований


**В исследовании:**
- Произведем автоматизированную загрузку в рабочую папку предоставленных таблиц
- Выясним, по какому признаку возможно объединение
- Для каждого принятого признака произведем:
 - загрузку файлов в тетрадку Юпитера или Коллаба
 - обработку данных
 - объединение файлов и получение сводного датафрейма
 - выгрузку сводных данных в рабочую папку в формате Excel
- Проведем анализ полученных сводных данных.

**Подготовим для заказчика скрипт для автоматизированного использования кода**

**Ход исследования:**
- Загрузка данных из предоставленных файлов и подготовка их к анализу
- Обработка и объединение данных в сводные таблицы
- Изучение общей информации о датафрейме и предобработка данных
- Исследовательский анализ данных
- Выводы и заключения


## Загрузка и подготовка данных к анализу


**Путь к файлу**

[Ссылка на данные](https://disk.yandex.ru/d/lSgVpGltZ4xIgQ).



Изучим данные и выполним предобработку. Есть ли в данных пропуски и дубликаты? Убедимся, что типы данных во всех колонках соответствуют сохранённым в них значениям.

Загрузим библиотеку Яндекс.Диск

In [193]:
pip install yadisk --quiet

Note: you may need to restart the kernel to use updated packages.


### Импортируем библиотеки:
- **pandas**  
- **sys**
- **IPython.display** для отображения данных
- **warnings** для отключения предупреждений
- **yadisk** для работы с данными Яндекс.Диска
- **os** для управления папками с данными
- **tqdm** для корректного отображения данных
- **openpyxl** для работы с таблицами Excel
- **logging** - для отображения логов
- **tqdm** - для отображения прцесса вычисления кода


In [194]:
# импортируем библиотеки
import pandas as pd
import sys
from IPython.display import display

import os
from tqdm import tqdm
from openpyxl import load_workbook
import warnings
from openpyxl import load_workbook
#pip install openpyxl
from openpyxl.cell.text import Font, InlineFont
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import yadisk
import logging

Перейдем в рабочую область

In [195]:
os.chdir('/home/jovyan/work')

Введем идентификационные данные для подключения

In [196]:
from cfg import app_id, secret_id, ya_token

Загрузим словарь `regions_in_federal`, в котором в качестве ключей укажем наименования федеральных округов, а в качестве значений - наименования входящих в этот федеральный округ регионов

In [197]:
from fo_regions import regions_in_federal

Получим токен для доступа к Яндекс,диску

In [198]:
# получаем токен для доступа к яндекс диску


y = yadisk.YaDisk(app_id, secret_id, ya_token)

if y.check_token():
    print("Sucessfully received token!")
else:
    print("Something went wrong. Not sure how though...")


Sucessfully received token!


Создадим список файлов для загрузки с Яндекс.Диска

In [199]:
# создаем список файлов для загрузки
list_of_files = []
for el in list(y.listdir('file_processor')):
    if el['path'].endswith('.xlsx'):
        list_of_files.append(el['path'])
print("\033[34m""\033[1m""Создан список файлов list_of_files")

Создан список файлов list_of_files


### Для отслеживания работы кода создадим файл `mylog.log`, куда будем сохранять логи.   


In [200]:
logging.basicConfig(
    level=logging.DEBUG, 
    filename = "mylog.log", 
    format = "%(asctime)s - %(module)s - %(levelname)s - %(funcName)s: %(lineno)d - %(message)s", 
    datefmt='%H:%M:%S',
    )

logging.info('info')
logging.debug('debug')
logging.warning('warning_message')

### Загрузка файлов в рабочую папку

Создадим в рабочей области папку для загрузки файлов с Яндекс.Диска и перейдем в нее

In [201]:
# создаем папку загрузки
load_path = 'test_files'
if not os.path.exists(load_path):
    os.mkdir(load_path)
os.chdir(load_path)

Загрузим (скопируем) файлы с Яндекс.Диска в созданную папку в рабочей области

In [202]:
# загружаем файлы с яндекс диска
for file in tqdm(list_of_files):
    y.download(file.split(':')[1], file.split('/')[-1])
    
print("\033[34m""\033[1m""Файлы загружены в папку в рабочей области test_files")    

100%|██████████| 147/147 [01:06<00:00,  2.20it/s]

Файлы загружены в папку в рабочей области test_files


Перейдем в рабочую папку

In [203]:
os.chdir(path="/home/jovyan/work/test_files")


### Обработка первой группы файлов.
Обработаем файлы, содержащие Сведения о лечении злокачественных новообразований (ЗНО), впервые зарегистрированных в 2021 г. подлежащих радикальному лечению.

#### Подготовительные действия

Cоздадим список `files_list_one`, содержащий в себе изменяющуюся часть названий файлов, удовлетворяющих условиям:
- Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
- Содержание файла: "Сведения о лечении злокачественных новообразований (ЗНО), впервые зарегистрированных в 2021 г. подлежащих радикальному лечению "  
- где `XXX` - в диапазоне: 058, 061-085

In [204]:
files_list_one = [58] + list(range(61, 86))

Преобразуем список - приведем его элементы к трех-значному виду.   
Создадим для этого функцию:
- `reload_files_list` - наполнит список трех-значными значениями


In [205]:
def reload_files_list (list_name):

    return(['00' + str(x) if len(str(x)) == 1  \
                  else '0' + str(x) if len(str(x)) == 2 \
                  else str(x) \
                  for x in list_name])

In [206]:
files_list_one = reload_files_list(files_list_one)

Создадим функцию `list_file_names`, которая создаст и заполнит список `convert_one` (и другие списки) названиями файлов, удовлетворяющих задаваемым условиям:


In [207]:
def list_file_names(list_name):
    os.chdir('/home/jovyan/work/test_files')
    #print("\033[34m"'Текущая папка: '"\033[1m", os.getcwd())
    convert_one = []
    for file in os.listdir():
        if not 'checkpoints' in file:
            if str(file.split('_')[2]) in list_name and 'Сос' in file:
                convert_one.append(file)
            else:
                pass
    return(convert_one)

Вызовем функцию `list_file_names` и создадим список `convert_one` с названиями файлов, удовлетворяющих условиям:
- Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
- Содержание файла: "Сведения о лечении злокачественных новообразований (ЗНО), впервые зарегистрированных в 2021 г. подлежащих радикальному лечению "  
- где `XXX` - в диапазоне: 058, 061-085

In [208]:
convert_one = list_file_names(files_list_one)

#### Создание сводного датафрейма

Создадим для последующего заполнения пустой список `final_one`

In [209]:
final_one = []

Заполним список `final_one` названиями файлов, удовлетворяющих условиям:
- Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
- Содержание файла: "Сведения о лечении злокачественных новообразований (ЗНО), впервые зарегистрированных в 2021 г. подлежащих радикальному лечению "  
- где `XXX` - в диапазоне: 058, 061-085

In [210]:
for el in tqdm(convert_one):
    vars = load_workbook(el).active['A1'].value.split('  ')

    ind, loc = ' '.join(vars[0].split('\n')[0:2]).capitalize(), vars[0].split('\n')[2]
    table = vars[-1].strip()
    df = pd.read_excel(el, header=[2])
    df.columns = ['Регион',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году, чел.',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году, % от впервые выявленных',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых будет продолжено в отчетом году, чел.',
                  'Число ЗНО, выявленных в отчетном году, радикальное лечение которых будет продолжено в отчетном году, % от впервые выявленных',
                  'В том числе с использованием методов только хирургического, %',
                  'В том числе с использованием методов только лучевого, %',
                  'В том числе с использованием методов только лекарственного, %',
                  'В том числе с использованием методов комбинированного или комплексного (кроме химиолучевого), %',
                  'В том числе с использованием методов химиолучевого, %']
    df['Индикатор'] = ind
    df['Локализация'] = loc
    df['Таблица'] = table
    df['Тип данных'] = 'СОП'
    final_one.append(df)

100%|██████████| 26/26 [00:04<00:00,  5.26it/s]


In [211]:
final_one_df = pd.concat(final_one).reset_index(drop=True)

В переменную `bad_regions` сохраним список значений из столбца `Регион`, обозначающих наименования федеральных округов

In [212]:
bad_regions = final_one_df[final_one_df.Регион.str.upper().str.contains('ФО')]['Регион'].tolist()

Для дальнейшего анализа создадим датафрейм `data_federal_district`: сохраним в нем строки (записи) , в которых в толбце `Регион` присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной `bad_regions`)

In [213]:
data_federal_district = final_one_df[final_one_df.Регион.isin(bad_regions)]

Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной `bad_regions`)

In [214]:
final_one_df = final_one_df[~final_one_df.Регион.isin(bad_regions)]

Произведем категоризацию данных по наименованию региона и соответствующие наименования федеральных округов запишем в столбец `ФО`.

In [215]:
def categorize_purpose(total_income):
    for key, value in regions_in_federal.items():
        if total_income in value:
            return key

final_one_df['ФО'] = final_one_df['Регион'].apply(categorize_purpose)

Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона

In [216]:
cols = final_one_df.columns.tolist()
cols = cols[:1] + cols[-1:]+ cols[1:-1]
final_one_df = final_one_df[cols].reset_index(drop = True)
#final_one_df

#### Выгрузка сводного файла в рабочую папку

Создадим шрифт для заголовка таблицы Excel

In [217]:
font = Font(name='Calibri',
             size=11,
            bold=False,
             italic=False,
             vertAlign=None,
             underline='none',
             strike=False,
             color='00FF0000')
inline_font = InlineFont(font)

Создадим функцию `df_download`для выгрузки файлов в форматах:
- .csv
- .xlsx

Выгрузим файл в папку в формате Excel  
Воспользуемся функцией `to_excel()` , для чего сначала установим `openpyxl`, чтобы мы могли записывать файлы в Excel.  
Итоговый файл также выгрузим в корневую папку

In [218]:
def df_download(data_name, df_name_xlsx, df_name_csv):

    # Сменим папку, перейдем в папку для хранения обработанных файлов
    os.chdir('/home/jovyan/work')
    print("\033[34m"'Текущая папка: '"\033[1m", os.getcwd())

    # Выгрузим файл в корневую папку в формате .csv
    final_one_df.to_csv(df_name_csv)
    print("\033[34m"'Выгружен файл'"\033[1m", df_name_csv)

    # Выгрузим файл в папку в формате Excel
    # Воспользуемся функцией to_excel() , для чего ранее мы установили openpyxl, чтобы записывать файлы в Excel.
    # Итоговый файл также выгрузим в корневую папку

    workbook = Workbook()
    sheet = workbook.active

    for row in dataframe_to_rows(data_name, header=True):
        sheet.append(row)

    # изменим высоту первой строки
    sheet.row_dimensions[1].height = 50

    # изменим ширину некоторых колонок
    columns_to_change = ['B', 'C', 'M', 'N', 'O']
    for cell in columns_to_change:
        sheet.column_dimensions[cell].width = 20

    # свернем строки с номерами с 3 по 94
    #sheet.row_dimensions.group(3, 95, hidden=False)
    #sheet.row_dimensions.group(97, 189, hidden=False)
    #sheet.row_dimensions.group(191, 283, hidden=False)
    #sheet.row_dimensions.group(285, 373, hidden=False)


    # установим жирный шрифт для ячеек первой строки
    cell_to_change = ['A1','B1','C1','D1','E1','F1','G1','H1','I1', 'J1','G1','K1','L1','M1','N1','O1','P1',]
    for cell in cell_to_change:
        sheet[cell].font = Font(size=12, bold=True)

    # удалим 2 строку (пустую)
    sheet.delete_rows(2)

    # зафиксируем первую строку при прокрутке по вертикали
    sheet.freeze_panes = 'A2'

    # выгрузим таблицу в рабочую область
    workbook.save(df_name_xlsx)
    print("\033[34m"'Выгружен файл'"\033[1m", df_name_xlsx)

    return

In [219]:
df_download(final_one_df, "final_one_SOP.xlsx", 'final_one_SOP.csv')

Текущая папка:  /home/jovyan/work
Выгружен файл final_one_SOP.csv
Выгружен файл final_one_SOP.xlsx


### Обработка второй группы файлов.
Обработаем файлы, содержащие Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г.

#### Подготовительные действия

Cоздадим список `files_list_two`, содержащий в себе изменяющуюся часть названий файлов, удовлетворяющих условиям:
- Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
- Содержание файла: "Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г."  
- где `XXX` - в диапазоне: 024, 027-051

In [220]:
files_list_two  = [24] + list(range(27, 52))

Создадим функцию `two_in_one`, которая вызовет посследовательно две ранее созданные функции:
- `reload_files_list`,  которая преобразует список `files_list_two`, указав в нем трех-значные значения
- `list_file_names`, которая создаст список `convert_two` с названиями файлов, удовлетворяющих условиям:
  - Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
  - Содержание файла: "Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г."  
  - где `XXX` - в диапазоне: 024, 027-051

In [221]:
def two_in_one(list_name):
    # Применим созданную ранее функцию reload_files_list, которая преобразуем список files_list_two,
    # указав в нем трех-значные значения
    list_name = reload_files_list(list_name)
    
    # Вызовем функцию list_file_names и создадим список convert_two
    convert_two = list_file_names(list_name)
    return(convert_two)

Создадим список `convert_two`

In [222]:
convert_two = two_in_one(files_list_two)

##### Проверим в каждом файле:
- какие листы присутствуют в файле
- какой лист является первым
- какой лист является активным

In [223]:
'''for el in tqdm(convert_two):
    wb2 = load_workbook(el)
    print('наименования листов', wb2.sheetnames)
    print('******************')
    #t = wb2.active.tolist()
    if wb2.sheetnames[0] in wb2.active:
        print('хорошо')
        '''

"for el in tqdm(convert_two):\n    wb2 = load_workbook(el)\n    print('наименования листов', wb2.sheetnames)\n    print('******************')\n    #t = wb2.active.tolist()\n    if wb2.sheetnames[0] in wb2.active:\n        print('хорошо')\n        "

В каждом файле:
- по два листа
- для последнего файла (файла с номером 051) листы называются ['Table 2', 'Table 3']
- для всех остальных файлов - листы называются ['Table 1', 'Table 3']
- в первом файле (файле с номером 024) активным является второй лист
- во всех остальных файлах активным является первый лист

##### Зададим, чтобы при чтении файла с номером 024 -  `2021_Таблица_024_Состояние_онко_помощи_в_РФ.xlsx` считывался (был активным) первый лист в файле

In [224]:
wb10 = load_workbook('2021_Таблица_024_Состояние_онко_помощи_в_РФ.xlsx')
print(wb10.sheetnames)

['Table 1', 'Table 3']


In [225]:
wb10.active = wb10['Table 3']

In [226]:
sheetss = wb10.sheetnames
ws = wb10[sheetss[0]]
ws

<Worksheet "Table 1">

##### Выведем на экран первые строки файлов, содержащихся в списке `convert_two`

In [227]:
'''
for el in tqdm(convert_two):
    display(pd.read_excel(el).head())
    '''

'\nfor el in tqdm(convert_two):\n    display(pd.read_excel(el).head())\n    '

Мы видим:

- Таблицы `2021_Таблица_024_Состояние_онко_помощи_в_РФ` и `2021_Таблица_051_Состояние_онко_помощи_в_РФ` содержат по 9 столбцов
- Остальные в диапазоне `files_list_two  = [24] + list(range(27, 52))` содержат по 10 столбцов. При этом последний – десятый - пустой
- Данные файла, находящегося между файлами с номерами 031 и 033 не имеет указания в себе номера таблицы. Соотнеся со списком `convert_two` установим, что это файл `2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx`
-

Выведем на экран значение последнего элемента в строке ячейки 'A1'

In [228]:
'''
for el in tqdm(convert_two):
    vars = load_workbook(el).active['A1'].value.split('  ')
    print('vars[-1] = ',vars[-1])
    '''

"\nfor el in tqdm(convert_two):\n    vars = load_workbook(el).active['A1'].value.split('  ')\n    print('vars[-1] = ',vars[-1])\n    "


Увидим, что:
- в файлах с номерами с 27 по 31, с 39 по 42, с 45 по 47, с 49 по 51 (список `files_list_two_alpha  = list(range(27, 32)) + list(range(39, 43)) + list(range(45, 48)) + list(range(49, 52))`) последний элемент содержит указание на номер таблицы
- в файле с номером 024 - указано "Продолжение таблицы 24" - вероятно, это второй (а не первый) лист таблицы
- в файле с номером 032 - не указан номер таблицы (уже отмечали выше)
- в файлах с номерами с 33 по 38, с 43 по 44 и в 48 (список `files_list_two_beta  = list(range(33, 39)) + list(range(43, 45)) + [48]`) последний элемент содетжит указание на номер таблицы и локализацию заболевания одновременно

Выведем на экран значение первого и второго элементов в строке ячейки 'A1'

In [229]:
'''
for el in tqdm(convert_two):
    vars = load_workbook(el,'Table 1').active['A1'].value.split('  ')
    print(vars[0])
    print(vars[1])
'''


"\nfor el in tqdm(convert_two):\n    vars = load_workbook(el,'Table 1').active['A1'].value.split('  ')\n    print(vars[0])\n    print(vars[1])\n"

Увидим, что в первом файле - с номером 024 - значение в ячейке не соответствует значениям в ячейке в других файлах - при его обработке необходимо сменить активный лист

<div style="border-radius: 45px;border:solid green 7px; padding: 40px">
    
#### Вывод по Подготовке данных    

Таким образом перед объединением файлов списка `convert_two` нам нужно учеть следующее:
    
- файл 024 - указано "Продолжение таблицы 24" - вероятно, это второй (а не первый) лист таблицы
 - сменить активный лист
    - смена наименования листа таблицы    
 - обработать ячейку "А1" - в ней первый и второй элемен отличаются от аналогичных в других файлах
    - удалить пробелы
  
- удалить последний - пустой - столбец в файлах, где количество столюцов равно 10
    
- разъединить слово Таблица и наименование локазизации - там, где они представлены одним элементом
   
- в файле с номером 032 - дополнить номером таблици


#### Создание сводного датафрейма

##### Отдельно обработаем файл с номером 24 (список `files_list_two_delta  = [24]`)
Подготовим данные из этого файла отдельно


In [230]:
files_list_two  = [24]

In [231]:
convert_two = two_in_one(files_list_two)
#convert_two

###### Выведем на экран первые строки файла, содержащихся в списке `convert_two`

In [232]:
for el in convert_two:
    display(pd.read_excel(el).head())

,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,\nСОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные новообразования - всего (С00-96) Таблица 24",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%"
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN
2,РОССИЯ,490588,24.1,3940529,2690.5,2262078,57.4,8,5.2
3,ЦЕНТРАЛЬНЫЙ ФО,129871,24.1,1147377,2916.4,666555,58.1,8.8,4.5
4,Белгородская область,6134,27.4,44114,2854.9,24836,56.3,7.2,5.1


Увидим здесь дополнительный (по сравнению с другими файлами) разделитель строки `/n` после слова "НОВООБРАЗОВАНИЯМИ,".   
Трансформируем код с учетом этого наблюдения


Но проверим сначала - какой лист в файле активный в настоящий момент

In [233]:
for el in convert_two:
    wb2 = load_workbook(el)
    print('наименования листов', wb2.sheetnames)
    print('первый  лист', wb2.sheetnames[0])
    print('активный лист',wb2.active)

наименования листов ['Table 1', 'Table 3']
первый  лист Table 1
активный лист <Worksheet "Table 3">


При загрузке данных нам понадобится указать, что загрузку произвести с первого листа.  



Не будем торопиться и посмотрим, что находится в ячейке "А1" первого листа (листа 'Table 1')

In [234]:
for el in convert_two:
    vars = load_workbook(el)['Table 1']['A1'].value.split('  ')
    print(vars)
    print(len(vars))
    for i in range(len(vars)):
        try:
            vars.remove('')
            vars.remove(' ')
        except:
            t10=100
    print(vars)
    print(vars[0])
    print(vars[-1])

['СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,\nСОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные новообразования - всего (С00-96)', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ' Таблица 24', '', '', '', '', '', '', '', ' ']
32
['СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,\nСОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные новообразования - всего (С00-96)', ' Таблица 24']
СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,
СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.
Злокачественные новообразования - всего (С00-96)
 Таблица 24


<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
    
    Опс!
    
    в файле '2021_Таблица_024_Состояние_онко_помощи_в_РФ.xlsx' после указания номера таблицы следует еще 8 пробелов!  
    
    ['СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,\nСОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные новообразования - всего (С00-96)', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ' Таблица 24', '', '', '', '', '', '', '', ' ']
    

###### Импортируем файлы (данные) в `pandas`

In [235]:
final_two = []
for el in convert_two:
    vars = load_workbook(el)['Table 1']['A1'].value.split('  ')
    for i in range(len(vars)):
        try:
            vars.remove('')
            vars.remove(' ')
        except:
            t10=100
    ind = (vars[0].split('\n')[0] + vars[0].split('\n')[1]).lower().capitalize()
    loc = vars[0].split('\n')[2]
    table = vars[-1].strip()
    df = pd.read_excel(el, header=[2])
    df.columns = ['Регион',
                'Взято на учет больных с впервые в жизни уст. диагнозом ЗНО',
                'в т.ч. вы- явлены активно,%',
                'Находились на учете на конец года, абс.число',
                'Находились на учете на конец года, на 100 тыс. населения',
                'из них 5 лет и более, абс.число',
                'из них 5 лет и более, %  от сост. на учете',
                'Индекс накопления контингентов',
                'Летальность,%'
                ]
    df['Индикатор'] = ind
    df['Локализация'] = loc
    df['Таблица'] = table
    df['Тип данных'] = 'СОП'
    final_two.append(df)

##### Обработаем данные из остальных файлов - с номерами от 27 по 51


In [236]:
files_list_two  = list(range(27, 52))

In [237]:
convert_two = two_in_one(files_list_two)

Импортируем файлы (данные) в `pandas`  
Не обновляем список `final_two`, поскольку в нем уже находятся данные из файла с номеровм 24

In [238]:

for el in tqdm(convert_two):
    vars = load_workbook(el).active['A1'].value.split('  ')
    ind = (vars[0] + ' ' + vars[1].split('\n')[0]).lower().capitalize()

    # в зависимости от заполенения ячейки "А1" присвоим значения переменным loc и table
    try:
        loc = vars[1].split('\n')[1]
        #print('loc = ', loc)
        if "Таблица " in vars[-1]:
            #print('таблица есть')
            table = vars[-1].strip()
            #print('table = ', table)
        else:
            table = 'Таблица 32'
            #print('table = ', table)
    except:
        loc = vars[-1].split('\n')[1].strip()
        table = vars[-1].split('\n')[0].strip()

    df = pd.read_excel(el, header=[2])
    # Если присутствует десятый столбец - удалим его
    if 'Unnamed: 9' in df.columns:
        df = df.drop(columns=['Unnamed: 9'])
    df.columns = ['Регион',
                'Взято на учет больных с впервые в жизни уст. диагнозом ЗНО',
                'в т.ч. вы- явлены активно,%',
                'Находились на учете на конец года, абс.число',
                'Находились на учете на конец года, на 100 тыс. населения',
                'из них 5 лет и более, абс.число',
                'из них 5 лет и более, %  от сост. на учете',
                'Индекс накопления контингентов',
                'Летальность,%'#,
                  #'ПУСТОЙ'
                ]
    df['Индикатор'] = ind
    df['Локализация'] = loc
    df['Таблица'] = table
    df['Тип данных'] = 'СОП'
    final_two.append(df)

100%|██████████| 25/25 [00:05<00:00,  4.71it/s]


Объединим только что полученные данные с данными из файла 24

In [239]:
final_two_df = pd.concat(final_two).reset_index(drop=True)

Ознакомимся с информацией в полученном датафрейме:

- всего в датафрейме 2444 записей
- пропущенные значения отсутствуют
- явные дубоикаты отсутствуют


In [240]:
final_two_df.isna().sum()

Регион                                                        0
Взято на учет больных с впервые в жизни уст. диагнозом ЗНО    0
в т.ч. вы- явлены активно,%                                   0
Находились на учете на конец года, абс.число                  0
Находились на учете на конец года, на 100 тыс. населения      0
из них 5 лет и более, абс.число                               0
из них 5 лет и более, %  от сост. на учете                    0
Индекс накопления контингентов                                0
Летальность,%                                                 0
Индикатор                                                     0
Локализация                                                   0
Таблица                                                       0
Тип данных                                                    0
dtype: int64

In [241]:
final_two_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2444 entries, 0 to 2443
Data columns (total 13 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Регион                                                      2444 non-null   object 
 1   Взято на учет больных с впервые в жизни уст. диагнозом ЗНО  2444 non-null   int64  
 2   в т.ч. вы- явлены активно,%                                 2444 non-null   float64
 3   Находились на учете на конец года, абс.число                2444 non-null   int64  
 4   Находились на учете на конец года, на 100 тыс. населения    2444 non-null   float64
 5   из них 5 лет и более, абс.число                             2444 non-null   int64  
 6   из них 5 лет и более, %  от сост. на учете                  2444 non-null   float64
 7   Индекс накопления контингентов                              2444 non-null   float64
 8 

In [242]:
final_two_df.duplicated().sum()

0

##### "Почистим" полученный датафрейм
- Для дальнейшего анализа создадим датафрейм data_federal_district_two: сохраним в нем строки (записи) , в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной bad_regions)
- Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной `bad_regions`)
- Произведем категоризацию данных по наименованию региона и соответствующие наименования федеральных округов запишем в столбец ФО.
- Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона

запишем действия в функцию `clear_df`.

In [243]:
def clear_data(data_name):

    # создадим датафрейм data_federal_district_two: сохраним в нем строки (записи),
    # в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов
    data_federal_district = data_name[data_name.Регион.isin(bad_regions)]

    # Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов
    data_name = data_name[~data_name.Регион.isin(bad_regions)]

    # Произведем категоризацию данных по наименованию региона и соответствующие наименования
    # федеральных округов запишем в столбец ФО.
    data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)

    # Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона
    cols = data_name.columns.tolist()
    cols = cols[:1] + cols[-1:]+ cols[1:-1]
    final_df = data_name[cols]#.reset_index(drop = True)
    final_df

    return(data_federal_district, final_df)

In [244]:
data_federal_district_two = clear_data(final_two_df)[0]
final_two_df = clear_data(final_two_df)[1]

/tmp/ipykernel_48/28608235.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)


In [245]:
#final_two_df = final_two_df

#### Выгрузка сводного файла в рабочую папку

In [246]:

df_download(final_two_df, "final_two_SOP.xlsx", "final_two_SOP.csv")

Текущая папка:  /home/jovyan/work
Выгружен файл final_two_SOP.csv
Выгружен файл final_two_SOP.xlsx


### Обработка второй группы файлов - вторые листы.
Обработаем файлы, содержащие Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г - - вторые листы.

#### Подготовительные действия

Вспомним, что на этом этапе обрабатываемые файлы: с названиями, удовлетворяющих условиям:
  - Название файла: `2021_Таблица_ХХХ_Состояние_онко_помощи_в_РФ.xlsx`  
  - Содержание файла: "Сведения о контингенте больных со злокачественными новообразованиями, состоящем на учете в онкологических учреждениях в 2021 г."  
  - где `XXX` - в диапазоне: 024, 027-051
  - ВТОРЫЕ листы

- `reload_files_list`,  которая преобразует список `files_list_two`, указав в нем трех-значные значения
- `list_file_names`, которая создаст список `convert_two` с названиями файлов, удовлетворяющих условиям:


Используем функцию `two_in_one` для списка `files_list_two  = [24] + list(range(27, 52))`

In [247]:
files_list_two  = [24] + list(range(27, 52))

Создадим список `convert_two`

In [248]:
convert_two = two_in_one(files_list_two)

##### Проверим в каждом файле:
- какие листы присутствуют в файле
- какой лист является первым
- какой лист является активным

In [249]:
'''
for el in tqdm(convert_two):
    wb2 = load_workbook(el)
    print('наименования файла', el.split('_')[2])
    print('наименования листов', wb2.sheetnames)
    print('первый  лист', wb2.sheetnames[0])
    print('активный лист',wb2.active)
    print('******************')
    #t = wb2.active.tolist()
    if wb2.sheetnames[0] in wb2.active:
    #if 'Table' in wb2.active:
        print('хорошо')
        '''

"\nfor el in tqdm(convert_two):\n    wb2 = load_workbook(el)\n    print('наименования файла', el.split('_')[2])\n    print('наименования листов', wb2.sheetnames)\n    print('первый  лист', wb2.sheetnames[0])\n    print('активный лист',wb2.active)\n    print('******************')\n    #t = wb2.active.tolist()\n    if wb2.sheetnames[0] in wb2.active:\n    #if 'Table' in wb2.active:\n        print('хорошо')\n        "

Вспомним, что в файле в номером 51 второй лист называется `'Table 5'`.  
В остальных файлах второй лист называется `'Table 3'`.  
Переименуем второй лист и назовем его `'Table 3'`.  
Сохраним файл под прежним именем.  
Выведем на экран наименования вторых листов.

In [250]:
for el in tqdm(convert_two):
    ss = load_workbook(el)
    #print('номер файла', el.split('_')[2])
    #print('Второй  лист', ss.sheetnames[1])
    #print('******************')
    if ss.sheetnames[1] ==  'Table 5':
        #print('выявлен "Table 5"')
        ss_sheet = ss.get_sheet_by_name('Table 5')
        ss_sheet.title = 'Table 3'
        ss.save(el)
        ss = load_workbook(el)
        #print('Лист Переименован')
        #print('второй  лист', wb2.sheetnames[1])
        #print('******************')


 96%|█████████▌| 25/26 [00:03<00:00, 10.07it/s]/tmp/ipykernel_48/435653300.py:8: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ss_sheet = ss.get_sheet_by_name('Table 5')
100%|██████████| 26/26 [00:03<00:00,  7.07it/s]


##### Выведем на экран первые строки файлов, содержащихся в списке `convert_two`

In [251]:
'''for el in tqdm(convert_two):
    display(pd.read_excel(el, 'Table 3').head())
  '''

"for el in tqdm(convert_two):\n    display(pd.read_excel(el, 'Table 3').head())\n  "

Мы видим:

- Таблицы `2021_Таблица_024_Состояние_онко_помощи_в_РФ` и `2021_Таблица_051_Состояние_онко_помощи_в_РФ` содержат по 9 столбцов
- Остальные в диапазоне `files_list_two  = [24] + list(range(27, 52))` содержат по 10 столбцов. При этом последний – десятый - пустой
- Данные файла, находящегося между файлами с номерами 031 и 033 не имеет указания в себе номера таблицы. Соотнеся со списком `convert_two` установим, что это файл `2021_Таблица_032_Состояние_онко_помощи_в_РФ.xlsx`
-

Выведем на экран значение некоторых элементов в строке ячейки 'A1'

In [252]:
'''for el in tqdm(convert_two):
    vars = load_workbook(el)['Table 3']['A1'].value.split('  ')
    ss = load_workbook(el)#['Table 3']
    print('номер файла = ',el.split('_')[2])
    print('Второй  лист', ss.sheetnames[1])
    print('vars[0] = ',vars[0])
    try:
        print('vars[1] = ',vars[1])
    except:
        print("\033[34m"'vars[1] отсутствует, в списке только один элемент'"\033[00m")
    try:
        print('vars[-2] = ',vars[-2])
    except:
        print("\033[34m"'vars[-2] отсутствует, в списке только один элемент'"\033[00m")

    print('vars[-1] = ',vars[-1])
    print('--------------------------------------------')
    '''

'for el in tqdm(convert_two):\n    vars = load_workbook(el)[\'Table 3\'][\'A1\'].value.split(\'  \')\n    ss = load_workbook(el)#[\'Table 3\']\n    print(\'номер файла = \',el.split(\'_\')[2])\n    print(\'Второй  лист\', ss.sheetnames[1])\n    print(\'vars[0] = \',vars[0])\n    try:\n        print(\'vars[1] = \',vars[1])\n    except:\n        print("\x1b"\'vars[1] отсутствует, в списке только один элемент\'"\x1b")\n    try:\n        print(\'vars[-2] = \',vars[-2])\n    except:\n        print("\x1b"\'vars[-2] отсутствует, в списке только один элемент\'"\x1b")\n\n    print(\'vars[-1] = \',vars[-1])\n    print(\'--------------------------------------------\')\n    '

<div style="border-radius: 45px;border:solid green 7px; padding: 40px">
    
#### Вывод по Подготовке данных    
- в файлах с номерами 29 и 51 заголовок второго листа НЕ совпадает с типовым заголовком остальных листов. При этом второй лист содержит данные, аналогичные данным остальных файлов – переименовать и сохранить заново
- в файлах с номерами 27 и 28 в ячейке «А1» присутствует только один элемент – в нем вся информация. В других файлах информация записана в разных элементах
- в части файлов номер таблицы находится в последнем элементе, в другой части – в предпоследнем
- в части элементов локализация заболевания находится в конце первого элемента, в другой части – в последнем элементе и совмещена со словом «Продолжение
- во всех файлах, кроме 051-го - присутствует 10-й - пустой - столбец




#### Создание сводного датафрейма

##### Обработаем файлы с номерами 29 и 51
В файлах с номерами 29 и 51 заголовок второго листа НЕ совпадает с типовым заголовком остальных листов. При этом второй лист содержит данные, аналогичные данным остальных файлов – переименуем и сохраним заново

In [253]:
files_list_two_second_list  = [29] + [51]

In [254]:
convert_to_change = two_in_one(files_list_two_second_list)

In [255]:
for el in tqdm(convert_to_change):
    vars = load_workbook(el)['Table 3']['A1'].value.split('  ')
    for i in range(len(vars)):
        try:
            vars.remove('')
            vars.remove(' ')
        except:
            t10=100

    if el in tqdm(convert_to_change):
        ind = ('ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ, ВЫЯВЛЕННЫХ В 2021 Г.').lower().capitalize()
        vars.remove('СВЕДЕНИЯ О')
    else:
        ind = (vars[0] + ' ' + vars[1].split('\n')[0]).lower().capitalize()

    print(vars)
    print(ind)



 50%|█████     | 1/2 [00:00<00:00,  9.36it/s]

['КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nГлотка (С10-13)', ' Таблица 29']
Показатели диагностики злокачественных новообразований, выявленных в 2021 г.



100%|██████████| 2/2 [00:00<00:00, 11.08it/s]

['КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЛейкемии (С91-95)', 'Таблица 51']
Показатели диагностики злокачественных новообразований, выявленных в 2021 г.


##### Обработаем данные (из  файлов - с номерами 24 и от 27 по 51)


In [256]:
files_list_two_2nd_sheet  = [24] + list(range(27, 52))

In [257]:
convert_two_2nd_sheet = two_in_one(files_list_two_2nd_sheet)

Импортируем файлы (данные) в `pandas`  
Не обновляем список `final_two`, поскольку в нем уже находятся данные из файла с номеровм 24

In [258]:
final_two = []

In [259]:

for el in convert_two:
    vars = load_workbook(el)['Table 3']['A1'].value.split('  ')
    # Удалим повторяющиеся пробелы
    for i in range(len(vars)):
        try:
            vars.remove('')
            vars.remove(' ')
        except:
            t10=100

    # в файлах с номерами 29 и 51 - заменим наименование таблицы
    if el in convert_to_change:
        ind = ('ПОКАЗАТЕЛИ ДИАГНОСТИКИ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ, ВЫЯВЛЕННЫХ В 2021 Г.').lower().capitalize()
        vars.remove('СВЕДЕНИЯ О')
    else:
        ind = (vars[0].split('\n')[0]).lower().capitalize()

    # в зависимости от заполенения ячейки "А1" присвоим значения переменной loc

    try:
        loc = vars[-1].split('\n')[1].strip()
        #print(el)
        #print('loc первая попытка = ', loc)

    except:
        loc = vars[0].split('\n')[1]
        #print(el)
        #print('loc вторая попытка = ', loc)

    # в зависимости от заполенения ячейки "А1" присвоим значения переменной table
    try:
        table = vars[0].split('\n')[3].strip()
        #print(el)
        #print('table первая попытка = ', table)

    except:
        try:
            table = vars[0].split('\n')[2].strip()
            #print(el)
            #print('table вторая попытка = ', table)
        except:
            try:
                table = vars[-1].split('\n')[0].strip()
                if table == "Продолжение":
                    table = vars[-2]
                    #print(el)
                    #print('table четвертая попытка = ', table)

                else:
                    #print(el)
                    #print('table третья попытка = ', table)
                    pass
            except:
                try:
                    table = vars[-2]
                    #print(el)
                    #print('table четвертая попытка = ', table)
                except:
                    table = vars[-1]
                    #print(el)
                    #print('table пятая попытка = ', table)


    df = pd.read_excel(el, header=[2])
    # Если присутствует десятый столбец - удалим его
    if 'Unnamed: 9' in df.columns:
        df = df.drop(columns=['Unnamed: 9'])
    df.columns = ['Регион',
                'Зарегистриро- вано ЗНО (без учтенных посмертно)',
                'из них: диагноз подтвержден морфологически, %',
                'имели стадию заболевания - I %',
                'имели стадию заболевания - II %',
                'имели стадию заболевания - III %',
                'имели стадию заболевания - IV %',
                'имели стадию заболевания - не установлена %',
                'Летальность на первом году с момента уст. диагноза, %'
                  ]
    df['Индикатор'] = ind
    df['Локализация'] = loc
    df['Таблица'] = table
    df['Тип данных'] = 'СОП'
    final_two.append(df)

Сформируем датафрейм

In [260]:
final_two_df_2nd_sheet = pd.concat(final_two).reset_index(drop=True)

Ознакомимся с информацией в полученном датафрейме:

- всего в датафрейме 2444 записей
- пропущенные значения отсутствуют
- явные дубликаты отсутствуют


In [261]:
final_two_df_2nd_sheet.isna().sum()

Регион                                                   0
Зарегистриро- вано ЗНО (без учтенных посмертно)          0
из них: диагноз подтвержден морфологически, %            0
имели стадию заболевания - I %                           0
имели стадию заболевания - II %                          0
имели стадию заболевания - III %                         0
имели стадию заболевания - IV %                          0
имели стадию заболевания - не установлена %              0
Летальность на первом году с момента уст. диагноза, %    0
Индикатор                                                0
Локализация                                              0
Таблица                                                  0
Тип данных                                               0
dtype: int64

In [262]:
final_two_df_2nd_sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2444 entries, 0 to 2443
Data columns (total 13 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Регион                                                 2444 non-null   object 
 1   Зарегистриро- вано ЗНО (без учтенных посмертно)        2444 non-null   int64  
 2   из них: диагноз подтвержден морфологически, %          2444 non-null   float64
 3   имели стадию заболевания - I %                         2444 non-null   int64  
 4   имели стадию заболевания - II %                        2444 non-null   float64
 5   имели стадию заболевания - III %                       2444 non-null   int64  
 6   имели стадию заболевания - IV %                        2444 non-null   float64
 7   имели стадию заболевания - не установлена %            2444 non-null   float64
 8   Летальность на первом году с момента уст. диагно

In [263]:
final_two_df_2nd_sheet.duplicated().sum()

0

##### "Почистим" полученный датафрейм
- Для дальнейшего анализа создадим датафрейм data_federal_district_two: сохраним в нем строки (записи) , в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной bad_regions)
- Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной `bad_regions`)
- Произведем категоризацию данных по наименованию региона и соответствующие наименования федеральных округов запишем в столбец ФО.
- Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона

запишем действия в функцию `clear_df`.

In [264]:
def clear_data(data_name):

    # создадим датафрейм data_federal_district_two: сохраним в нем строки (записи),
    # в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов
    data_federal_district = data_name[data_name.Регион.isin(bad_regions)]

    # Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов
    data_name = data_name[~data_name.Регион.isin(bad_regions)]

    # Произведем категоризацию данных по наименованию региона и соответствующие наименования
    # федеральных округов запишем в столбец ФО.
    data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)

    # Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона
    cols = data_name.columns.tolist()
    cols = cols[:1] + cols[-1:]+ cols[1:-1]
    final_df = data_name[cols]#.reset_index(drop = True)
    final_df

    return(data_federal_district, final_df)

In [265]:
data_federal_district_two = clear_data(final_two_df_2nd_sheet)[0]
final_two_df_2nd_sheet = clear_data(final_two_df_2nd_sheet)[1]

/tmp/ipykernel_48/28608235.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)


In [266]:
final_two_df_2nd_sheetfinal_two_df_2nd_sheet = final_two_df_2nd_sheet#.sort_values('Таблица')
#final_two_df_2nd_sheet.head()

#### Выгрузка сводного файла в рабочую папку

In [267]:

df_download(final_two_df_2nd_sheet, "final_two_SOP_second_part.xlsx", "final_two_SOP_second_part.csv")

Текущая папка:  /home/jovyan/work
Выгружен файл final_two_SOP_second_part.csv
Выгружен файл final_two_SOP_second_part.xlsx


### Обработка третьей группы файлов.
Обработаем файлы, содержащие Сведения о Злокачественных новообразованиях в РФ (заболеваемость и смертность) в 2021 г.

#### Подготовительные действия

Cоздадим список `files_list_three`, содержащий в себе изменяющуюся часть названий файлов, удовлетворяющих условиям:
- Название файла: `2021_Таблица_ХХХ_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx`  
- Содержание файла: "Заболеваемость населения территорий России злокачественными новообразованиями"  
- где `XXX` - в диапазоне: 012-059

In [268]:
files_list_three  = list(range(12, 59)) + list(range(69, 104))

Забегая вперед, создадим списки `files_list_gender_men_exception` и `files_list_gender_women_exception`, которые включат в себя номера файлов, содержащих локализацию заболеваний, характерных ислючительно для одного из полов.



In [269]:
files_list_gender_men_exception = list(range(42, 45)) + list(range(91, 93))

In [270]:
files_list_gender_women_exception = list(range(36, 42)) + list(range(87, 91))

Создадим функцию `two_in_one_ZNO`, которая вызовет последовательно две функции:
- ранее созданную функцию `reload_files_list`,  которая преобразует список `files_list_three`, указав в нем трех-значные значения
- новую функцию `list_file_names_ZNO`, которая создаст список `convert_three` с названиями файлов, удовлетворяющих условиям:
  - Название файла: `2021_Таблица_ХХХ_Злокачественные_новообразования_в_РФ_(заболеваемость_и_смертность).xlsx`  
  - Содержание файла: "Заболеваемость населения территорий России злокачественными новообразованиями"  
  - где `XXX` - в диапазоне: 012-059

Создадим функцию `list_file_names_ZNO`, которая создаст и заполнит список `convert_one` (и другие списки) названиями файлов, удовлетворяющих задаваемым условиям:


In [271]:
def list_file_names_ZNO(list_name):
    os.chdir('/home/jovyan/work/test_files')
    print("\033[34m"'Текущая папка: '"\033[1m", os.getcwd())
    convert_one = []
    for file in os.listdir():
        if not 'checkpoints' in file:
            if str(file.split('_')[2]) in list_name and 'Злокачественные' in file:
                convert_one.append(file)
            else:
                pass
    return(convert_one)

In [272]:
def two_in_one_ZNO(list_name):
    # Применим созданную ранее функцию reload_files_list, которая преобразует список files_list_two,
    # указав в нем трех-значные значения
    list_name = reload_files_list(list_name)

    # Вызовем функцию list_file_names_ZNO и создадим список convert_two
    convert_two = list_file_names_ZNO(list_name)

    return(convert_two)

Создадим список `convert_three`

In [273]:
convert_three = two_in_one_ZNO(files_list_three)

Текущая папка:  /home/jovyan/work/test_files


Создадим список `convert_gender_men_exception`

In [274]:
convert_gender_men_exception = two_in_one_ZNO(files_list_gender_men_exception)

Текущая папка:  /home/jovyan/work/test_files


Создадим список `convert_gender_women_exception`

In [275]:
convert_gender_women_exception = two_in_one_ZNO(files_list_gender_women_exception)

Текущая папка:  /home/jovyan/work/test_files


##### Выведем на экран первые строки файлов, содержащихся в списке `convert_two`

In [276]:
for el in convert_two:
    display(pd.read_excel(el).head(10))

,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ,\nСОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные новообразования - всего (С00-96) Таблица 24",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%"
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN
2,РОССИЯ,490588,24.1,3940529,2690.5,2262078,57.4,8,5.2
3,ЦЕНТРАЛЬНЫЙ ФО,129871,24.1,1147377,2916.4,666555,58.1,8.8,4.5
4,Белгородская область,6134,27.4,44114,2854.9,24836,56.3,7.2,5.1
5,Брянская область,5137,20.3,38258,3221.5,21775,56.9,7.4,6
6,Владимирская область,4986,10.6,44686,3309.4,26554,59.4,9,4.1
7,Воронежская область,8170,29.2,71536,3090.2,41212,57.6,8.8,5.1
8,Ивановская область,4129,28.9,31860,3211.4,18064,56.7,7.7,5.1
9,Тверская область,4812,20,39059,3117.2,23240,59.5,8.1,5.5


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nГуба (С00) Таблица 27",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,1675,35.4,32914,22.5,24985,75.9,19.7,0.9,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,308,41.6,6239,15.9,4679,75,20.3,1.2,NaN
4,Белгородская область,17,47.1,357,23.1,243,68.1,21,0.6,NaN
5,Брянская область,17,47.1,300,25.3,220,73.3,17.6,0.7,NaN
6,Владимирская область,16,12.5,275,20.4,201,73.1,17.2,1.8,NaN
7,Воронежская область,44,34.1,767,33.1,588,76.7,17.4,0.9,NaN
8,Ивановская область,19,47.4,189,19.1,132,69.8,9.9,0,NaN
9,Тверская область,15,20,292,23.3,239,81.8,19.5,0.7,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nПолость рта (C01-09) Таблица 28",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,8461,16.4,45166,30.8,23637,52.3,5.3,9.3,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,2347,15.6,13019,33.1,6610,50.8,5.5,8.8,NaN
4,Белгородская область,71,12.7,348,22.5,152,43.7,4.9,9.4,NaN
5,Брянская область,125,18.4,493,41.5,235,47.7,3.9,14.9,NaN
6,Владимирская область,92,1.1,483,35.8,213,44.1,5.3,8.3,NaN
7,Воронежская область,122,25.4,663,28.6,356,53.7,5.4,12.2,NaN
8,Ивановская область,77,10.4,393,39.6,200,50.9,5.1,11.5,NaN
9,Тверская область,103,8.7,476,38,249,52.3,4.6,12.3,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nГлотка (С10-13) Таблица 29",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,5097,9,18754,12.8,8137,43.4,3.7,15.3,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,1579,8.4,6317,16.1,2761,43.7,4,13.8,NaN
4,Белгородская область,106,5.7,321,20.8,122,38,3,13.9,NaN
5,Брянская область,113,11.5,242,20.4,70,28.9,2.1,25.1,NaN
6,Владимирская область,76,0,332,24.6,137,41.3,4.4,14,NaN
7,Воронежская область,149,5.4,477,20.6,185,38.8,3.2,16.5,NaN
8,Ивановская область,71,11.3,174,17.5,69,39.7,2.5,17.5,NaN
9,Тверская область,51,5.9,164,13.1,70,42.7,3.2,19.2,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nПищевод (С15) Таблица 30",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,6455,11,13876,9.5,5313,38.3,2.1,27.1,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,1491,10.3,3838,9.8,1490,38.8,2.6,22.3,NaN
4,Белгородская область,75,8,121,7.8,29,24,1.6,23.9,NaN
5,Брянская область,74,6.8,100,8.4,29,29,1.4,40.8,NaN
6,Владимирская область,72,4.2,201,14.9,84,41.8,2.8,23,NaN
7,Воронежская область,64,12.5,159,6.9,59,37.1,2.5,31.5,NaN
8,Ивановская область,65,18.5,75,7.6,14,18.7,1.2,39.5,NaN
9,Тверская область,61,6.6,128,10.2,36,28.1,2.1,32.6,NaN


...

,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nПочка (С64) Таблица 47",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,18306,23,194464,132.8,119456,61.4,10.6,3.3,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,4484,26.8,52576,133.6,32780,62.3,11.7,3.1,NaN
4,Белгородская область,234,14.5,2218,143.5,1319,59.5,9.5,3.1,NaN
5,Брянская область,188,13.8,1613,135.8,890,55.2,8.6,4.2,NaN
6,Владимирская область,193,10.9,2281,168.9,1456,63.8,11.8,2.6,NaN
7,Воронежская область,287,13.6,2933,126.7,1829,62.4,10.2,3,NaN
8,Ивановская область,158,32.9,1510,152.2,904,59.9,9.6,3.5,NaN
9,Тверская область,174,37.4,1858,148.3,1202,64.7,10.7,3.9,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г. Таблица 48\nМочевой пузырь (С67)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,13139,17.8,117188,80,66976,57.2,8.9,4,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,3496,19.3,34383,87.4,19474,56.6,9.8,3.7,NaN
4,Белгородская область,180,12.2,1551,100.4,897,57.8,8.6,3.1,NaN
5,Брянская область,145,12.4,1054,88.8,518,49.1,7.3,5.9,NaN
6,Владимирская область,144,4.9,1285,95.2,745,58,8.9,2.9,NaN
7,Воронежская область,233,10.7,2283,98.6,1327,58.1,9.8,3.1,NaN
8,Ивановская область,111,23.4,926,93.3,485,52.4,8.3,2.8,NaN
9,Тверская область,99,15.2,1177,93.9,763,64.8,11.9,3.1,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЩитовидная железа (С73) Таблица 49",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,11243,32.7,184556,126,130692,70.8,16.4,0.5,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,2991,28.6,52600,133.7,37819,71.9,17.6,0.4,NaN
4,Белгородская область,149,14.1,2108,136.4,1480,70.2,14.1,0.3,NaN
5,Брянская область,213,21.1,5571,469.1,4406,79.1,26.2,0.1,NaN
6,Владимирская область,102,20.6,1450,107.4,943,65,14.2,0.3,NaN
7,Воронежская область,170,36.5,3952,170.7,3054,77.3,23.2,0.3,NaN
8,Ивановская область,48,31.3,934,94.1,716,76.7,19.5,0.4,NaN
9,Тверская область,90,50,2486,198.4,2125,85.5,27.6,0.5,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЗлокачественные лимфомы (С81-86;88,90,96) Таблица 50",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%",NaN
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN,NaN
2,РОССИЯ,14087,8.3,137286,93.7,86555,63,9.7,4.3,NaN
3,ЦЕНТРАЛЬНЫЙ ФО,3617,10,41683,105.9,26078,62.6,11.5,3.6,NaN
4,Белгородская область,155,11,1583,102.4,926,58.5,10.2,3.2,NaN
5,Брянская область,161,5,1274,107.3,815,64,7.9,6,NaN
6,Владимирская область,125,2.4,1449,107.3,986,68,11.6,2.9,NaN
7,Воронежская область,204,3.4,2474,106.9,1587,64.1,12.1,4.4,NaN
8,Ивановская область,118,4.2,1082,109.1,695,64.2,9.2,4.4,NaN
9,Тверская область,133,6.8,1543,123.1,1113,72.1,11.6,4.3,NaN


,"СВЕДЕНИЯ О КОНТИНГЕНТЕ БОЛЬНЫХ СО ЗЛОКАЧЕСТВЕННЫМИ НОВООБРАЗОВАНИЯМИ, СОСТОЯЩЕМ НА УЧЕТЕ В ОНКОЛОГИЧЕСКИХ УЧРЕЖДЕНИЯХ В 2021 Г.\nЛейкемии (С91-95) Таблица 51",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"Республика, край, область",Взято на учет больных с впервые в жизни уст. д...,"в т.ч. вы- явлены активно,\n%",Находились на учете на конец года,NaN,из них 5 лет и более,NaN,Индекс на- копления контин- гентов,"Леталь- ность,\n%"
1,NaN,NaN,NaN,абсолют- ное число,на 100 тыс. населения,абсолют- ное число,% от сост. на учете,NaN,NaN
2,РОССИЯ,8636,8.2,87228,59.6,53584,61.4,10.1,4.8
3,ЦЕНТРАЛЬНЫЙ ФО,2344,8.4,26075,66.3,15707,60.2,11.1,4.1
4,Белгородская область,117,3.4,900,58.2,467,51.9,7.7,4.4
5,Брянская область,101,5.9,855,72,494,57.8,8.5,5.7
6,Владимирская область,59,0,604,44.7,399,66.1,10.2,4.9
7,Воронежская область,66,7.6,1302,56.2,932,71.6,19.7,4.4
8,Ивановская область,38,2.6,378,38.1,276,73,9.9,5
9,Тверская область,76,6.6,623,49.7,413,66.3,8.2,5.3





Мы видим:

- Таблицы с номерами от 36 до 44 содержат 10 столбцов
- остальные таблицы содержат по 13 столбцов
- различие объясняется тем, что Таблицы с номерами от 36 до 44 содержат информацию о локализациях заболеваний, характерную исключительно для одного из полов     

Посмотрим, в каких ячейках имеется информация и о чем она.  
Выведем на экран значение ячеек от 'A1' до 'В4'.  
Убедимся, что этой области нам достаточно для получения данных об информации на листе

In [277]:
'''for el in tqdm(convert_two):
    for j in ['A', 'B']:
        for i in range(1, 5):
            print(f'{j}{i}', end=', ')
            try:
                vars = load_workbook(el).active[f'{j}{i}'].value.split('  ')
                print('vars = ',vars)
            except:
                print("none")
    print('********************************************************************')
    #print('vars[-1] = ',vars[-1])
    '''

'for el in tqdm(convert_two):\n    for j in [\'A\', \'B\']:\n        for i in range(1, 5):\n            print(f\'{j}{i}\', end=\', \')\n            try:\n                vars = load_workbook(el).active[f\'{j}{i}\'].value.split(\'  \')\n                print(\'vars = \',vars)\n            except:\n                print("none")\n    print(\'********************************************************************\')\n    #print(\'vars[-1] = \',vars[-1])\n    '


Увидим, что:
- Информация о номере таблица всегда содержится в ячейке "А1"
- Информация о типе данных - всегда содержится в ячейке "А2". Иногда она выполнена полностью заглавными буквами
- Информация о локализации может содержаться:
 - в ячейке "А4" - и тогда ячейка "В4" пустая
 - в ячейке "В4" - и тогда в ячейке "А4" записана фраза "Локализация"


<div style="border-radius: 45px;border:solid green 7px; padding: 40px">
    
#### Вывод по Подготовке данных    
Таким образом перед объединением файлов списка `convert_two` нам нужно учеть следующее:
    
- Таблицы с номерами от 36 до 44 и от 87 до 92 содержат 10 столбцов - обрабатывать: только для обного из полов
- Информация о номере таблица всегда содержится в ячейке "А1"
- Информация о типе данных - всегда содержится в ячейке "А2". Иногда она выполнена полностью заглавными буквами
- Информация о локализации может содержаться:
 - в ячейке "А4" - и тогда ячейка "В4" пустая
 - в ячейке "В4" - и тогда в ячейке "А4" записана фраза "Локализация"




<div style="border-radius: 45px;border:solid red 7px; padding: 40px">
Для файлов ЗНО в диапазоне от 87 до 92 (соответствуют локализациям, характерным исключительно для одного из полов) Заголовок указан как:<br>
СМЕРТНОСТЬ НАСЕЛЕНИЯ РОССИИ ОТ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ  <br><br>
Для остальных файлов ЗНО со смертностью – заголовок указан как <br>
СМЕРТНОСТЬ НАСЕЛЕНИЯ ТЕРРИТОРИЙ РОССИИ ОТ ЗЛОКАЧЕСТВЕННЫХ НОВООБРАЗОВАНИЙ


#### Создание сводного датафрейма

Импортируем файлы (данные) в `pandas`  


In [278]:
final_three_total = []
final_three_men = []
final_three_women = []

В списки ниже отнесем номера столбцов, данные из которых возьмем для формирования соответственно списков по "Всему населению", мужчинам и женщинам

In [279]:

cols_total = [0, 1, 2, 3, 4]
cols_men = [0, 5, 6, 7, 8]
cols_women = [0, 9, 10, 11, 12]


Создадим список `col_new_names`, в который внесем новые названия столбцов

In [280]:
col_new_names = ['Регион',
                'Абсолютное Число',
                'Показатель на 100 тыс. населения "грубый"',
                'Показатель на 100 тыс. населения, стандартизованный',
                'Показатель на 100 тыс. населения, ошибка']

Если текущий файл попадает в список исключений для мужчин - то данные заносятся в файл для "Всего населения" и для мужчин; в файл для женщин попадают пропуски
Аналогично,
Если текущий файл попадает в список исключений для женщин - то данные заносятся в файл для "Всего населения" и для женщин; в файл для мужчин попадают пропуски

In [281]:
final_three_total = []
final_three_men = []
final_three_women = []

for el in tqdm(convert_three):

    #vars = load_workbook(el).active['A1'].value.split('  ')
    # Присвоим значения переменным Названия таблицы, номера таблицы и локализации заболевания
    ind = load_workbook(el).active['A2'].value.lower().capitalize()
    table = load_workbook(el).active['A1'].value
    try:
        loc = load_workbook(el).active['A4'].value.split(': ')[1]
        #print('first',loc)
    except:
        loc = load_workbook(el).active['B4'].value
        #print('second',loc)




    # Загрузим файлы в три переменных соответственно полу пациента
    df_total = pd.read_excel(el, header=[6], usecols=cols_total)
    #df_men = pd.read_excel(el, header=[6], usecols=cols_men)
    # Для списка файлов с локациями заболеваний, характерными исключительно для мужчин, предусмотрим два варианта
    if not el in convert_gender_men_exception:
        df_men = pd.read_excel(el, header=[6], usecols=cols_men)
    else:
        df_men = pd.read_excel(el, header=[6], usecols=cols_total)
    # Для списка файлов с локациями заболеваний, характерными исключительно для женщин, предусмотрим два варианта
    if not el in convert_gender_women_exception:
        if not el in convert_gender_men_exception:
            df_women = pd.read_excel(el, header=[6], usecols=cols_women)
        else:
            df_women = pd.read_excel(el, header=[6], usecols=cols_men)
    else:
        df_women = pd.read_excel(el, header=[6], usecols=cols_total)

    # присвоим столбцам новые названия
    df_total.columns = col_new_names
    df_men.columns = col_new_names
    df_women.columns = col_new_names

    list_gender = ['df_total', 'df_men', 'df_women']

    # Добавим столбцы с указанием Названия таблицы, номера таблицы, локализации заболевания и Типа принятых данных
    def complect(file_name):
        file_name['Индикатор'] = ind
        file_name['Локализация'] = loc
        file_name['Таблица'] = table
        file_name['Тип данных'] = 'ЗНО'

        return
    complect(df_total)
    complect(df_men)
    complect(df_women)

    # Добавим столбцы с указанием Пола пациента
    df_total['Пол пациента'] = 'Все население'
    df_men['Пол пациента'] = 'Мужчины'
    df_women['Пол пациента'] = 'Женщины'

    # Запишем данные в список `final_two`
    final_three_total.append(df_total)
    final_three_men.append(df_men)
    final_three_women.append(df_women)

100%|██████████| 82/82 [05:16<00:00,  3.86s/it]


##### Объединим полученные данные

In [282]:
final_df_total = pd.concat(final_three_total).reset_index(drop=True)

final_df_men = pd.concat(final_three_men).reset_index(drop=True)

final_df_women = pd.concat(final_three_women).reset_index(drop=True)

Ознакомимся с информацией в полученном датафрейме:


In [283]:
def check(data_name):

    print("Общая информация", data_name.info())
    print("Пропуски", data_name.isna().sum())
    print("Явные дубликаты", data_name.duplicated().sum())
    return

In [284]:
check(final_df_total)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7693 entries, 0 to 7692
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Регион                                               7693 non-null   object
 1   Абсолютное Число                                     7693 non-null   int64 
 2   Показатель на 100 тыс. населения "грубый"            7693 non-null   object
 3   Показатель на 100 тыс. населения, стандартизованный  7693 non-null   object
 4   Показатель на 100 тыс. населения, ошибка             7693 non-null   object
 5   Индикатор                                            7693 non-null   object
 6   Локализация                                          7693 non-null   object
 7   Таблица                                              7693 non-null   object
 8   Тип данных                                           7693 non-null   object
 9

In [285]:
check(final_df_men)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7693 entries, 0 to 7692
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Регион                                               7693 non-null   object 
 1   Абсолютное Число                                     6763 non-null   float64
 2   Показатель на 100 тыс. населения "грубый"            6763 non-null   object 
 3   Показатель на 100 тыс. населения, стандартизованный  6763 non-null   object 
 4   Показатель на 100 тыс. населения, ошибка             6763 non-null   object 
 5   Индикатор                                            7693 non-null   object 
 6   Локализация                                          7693 non-null   object 
 7   Таблица                                              7693 non-null   object 
 8   Тип данных                                           7693 non-null  

In [286]:
check(final_df_women)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7693 entries, 0 to 7692
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Регион                                               7693 non-null   object 
 1   Абсолютное Число                                     7228 non-null   float64
 2   Показатель на 100 тыс. населения "грубый"            7228 non-null   object 
 3   Показатель на 100 тыс. населения, стандартизованный  7228 non-null   object 
 4   Показатель на 100 тыс. населения, ошибка             7228 non-null   object 
 5   Индикатор                                            7693 non-null   object 
 6   Локализация                                          7693 non-null   object 
 7   Таблица                                              7693 non-null   object 
 8   Тип данных                                           7693 non-null  


- всего в каждом датафрейме 7693 записей
- явные дубоикаты в каждом датафрейме отсутствуют
- в датафрейме `final_df_total` пропущенные значения отсутствуют
- в датафреймах `final_df_men` и `final_df_women` пропущенные значения соответствуют локализациям заболеаний, характерным исключительно для одного из полов
- цифровые данные представлены не в числовом формате - необходимо устранить при дальнейшей обработке



##### Объединим полученные данные в единый  датафрейм
Объединим полученные три датафрейма в единый  датафрейм методом `concat`

In [287]:
final_df = pd.concat([final_df_total, final_df_men, final_df_women]).reset_index(drop=True)

##### "Почистим" полученный датафрейм и Выгрузим сводный файл в рабочую папку
- Для дальнейшего анализа создадим датафрейм data_federal_district_two: сохраним в нем строки (записи) , в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной bad_regions)
- Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов (присутствуют в переменной `bad_regions`)
- Произведем категоризацию данных по наименованию региона и соответствующие наименования федеральных округов запишем в столбец ФО.
- Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона

Запишем действия в функцию `clear_data_bad_reg_names`.

In [288]:
def clear_data_bad_reg_names(data_name):

    # создадим датафрейм data_federal_district_two: сохраним в нем строки (записи),
    # в которых в cтолбце Регион присутствуют значения, означающие наименования федеральных округов
    data_federal_district = data_name[data_name.Регион.isin(bad_regions)]

    # Удалим строки (записи) , в которых в столбце `Регион` присутствуют значения, означающие наименования федеральных округов
    data_name = data_name[~data_name.Регион.isin(bad_regions)]

    # Произведем категоризацию данных по наименованию региона и соответствующие наименования
    # федеральных округов запишем в столбец ФО.
    data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)

    # Изменим имеющийся порядок столбцов и поместим наименование округа после наименования региона, а после него - пол пациентов
    cols = data_name.columns.tolist()
    cols = cols[:1] +  cols[-1:]+ cols[1:-1]
    cols = cols[:2] +  cols[-1:]+ cols[2:-1]
    #cols = cols[:1] +  cols[-2:]+ cols[1:-2]
    final_df = data_name[cols]#.reset_index(drop = True)
    final_df

    return(data_federal_district, final_df)

In [289]:
# создадим список bad_regions: сохраним в нем наименования федеральных округов
final_df_total.Регион = final_df_total.Регион.str.strip()
bad_regions = final_df_total[final_df_total.Регион.str.upper().str.strip().str.contains('ФО')]['Регион'].tolist()

Подстроим известную нам функцию `df_download` под формат нового файла.  
Назовем ее `df_download_second`

In [345]:
def df_download_second(data_name, df_name_xlsx, df_name_csv):

    # Сменим папку, перейдем в папку для хранения обработанных файлов
    os.chdir('/home/jovyan/work')
    print("\033[34m"'Текущая папка: '"\033[1m", os.getcwd())

    # Выгрузим файл в корневую папку в формате .csv
    data_name.to_csv(df_name_csv)
    print("\033[34m"'Выгружен файл'"\033[1m", df_name_csv)

    # Выгрузим файл в папку в формате Excel
    # Воспользуемся функцией to_excel() , для чего ранее мы установили openpyxl, чтобы записывать файлы в Excel.
    # Итоговый файл также выгрузим в корневую папку

    workbook = Workbook()
    sheet = workbook.active

    for row in dataframe_to_rows(data_name, header=True):
        sheet.append(row)

    # изменим высоту первой строки
    sheet.row_dimensions[1].height = 50

    # изменим ширину некоторых колонок
    columns_to_change = ['B', 'C', 'D', 'I', 'J', 'K']
    for cell in columns_to_change:
        sheet.column_dimensions[cell].width = 20

    # свернем строки с номерами с 3 по 94
    #sheet.row_dimensions.group(3, 95, hidden=False)
    #sheet.row_dimensions.group(97, 189, hidden=False)
    #sheet.row_dimensions.group(191, 283, hidden=False)
    #sheet.row_dimensions.group(285, 373, hidden=False)


    # установим жирный шрифт для ячеек первой строки
    cell_to_change = ['A1','B1','C1','D1','E1','F1','G1','H1','I1', 'J1','G1','K1','L1','M1','N1','O1','P1',]
    for cell in cell_to_change:
        sheet[cell].font = Font(size=12, bold=True)

    # удалим 2 строку (пустую)
    sheet.delete_rows(2)

    # зафиксируем первую строку при прокрутке по вертикали
    sheet.freeze_panes = 'A2'

    # выгрузим таблицу в рабочую область
    workbook.save(df_name_xlsx)
    print("\033[34m"'Выгружен файл'"\033[1m", df_name_xlsx)

    return

Функция `wash_and_clean` сформирует и распечатает нам очищенные датафреймы для "Всего населения", мужчин и женщин

In [346]:
def wash_and_clean(data_name, data_xlsx, data_csv):
    # Удалим пробелы в первой и последней позиции в столбце "Регион" и "Индикатор"
    data_name.Регион = data_name.Регион.str.strip()
    data_name.Индикатор = data_name.Индикатор.str.strip()
    # Категоризируем регионы - зададим им названия ФО
    data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)
    # Сохраним данные в разрезе федеральных округов
    data_federal_district_two = clear_data_bad_reg_names(data_name)[0]
    # Сохраним датафрейм без строк с наименованиями ФО
    data_name = clear_data_bad_reg_names(data_name)[1]
    # выгрузим таблицы в формате Excel
    df_download_second(data_name, data_xlsx, data_csv)
    return(data_name)

Вызовем функцию `wash_and_clean` и сформируем и распечатаем датафреймы для "Всего населения", мужчин и женщин

In [347]:
df = wash_and_clean(final_df, "final_df_ZNO.xlsx", "final_df_ZNO.csv")

/tmp/ipykernel_48/391684453.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_name['ФО'] = data_name['Регион'].apply(categorize_purpose)


Текущая папка:  /home/jovyan/work
Выгружен файл final_df_ZNO.csv


  0%|          | 0/21111 [23:38<?, ?it/s]


Выгружен файл final_df_ZNO.xlsx
